## Using Python to Integrate MongoDB Data into an ETL Process
Modern Data Warehousing and Analytics solutions frequently use languages like Python or Scala to extract data from numerous sources, including relational database management systems, NoSQL database systems, real-time streaming endpoints and Data Lakes. These languages can then be used to perform many types of transformation before then loading the data into a variety of destinations including file systems and data warehouses. This data can then be consumed by data scientists or business analysts.

In this lab you will build upon the **Northwind_DW2** dimensional database from Lab 3; however, you will be integrating new data sourced from an instance of MongoDB. The new data will be concerned with new business processes; inventory and purchasing. You will continue to interact with both the source systems (MongoDB and MySQL), and the destination system (the Northwind_DW2 data warehouse) from a remote client running Python (Jupyter Notebooks). 

Just as in Lab 3, you will fetch data into Pandas DataFrames, perform all the necessary transformations in-memory on the client, and then push the newly transformed DataFrame to the RDBMS data warehouse using a Pandas function that will create the table and fill it with data with a single operation.

### Prerequisites:
This notebook uses the PyMongo database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install pymongo[srv]`

#### Import the Necessary Libraries

In [14]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

In [15]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 1.4.39
Running PyMongo Version: 4.6.3


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [16]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Passw0rd123",
    "hostname" : "localhost",
    "dbname" : "northwind_dw2"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "root",
    "password" : "Passw0rd123",
    "cluster_name" : "sandbox",
    "cluster_subnet" : "zibbf",
    "cluster_location" : "local", # "local"
    "db_name" : "northwind_purchasing"
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [17]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

In [18]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"suppliers" : 'northwind_suppliers.json',
              "invoices" : 'northwind_invoices.json',
              "purchase_orders" : 'northwind_purchase_orders.json',
              "inventory_transactions" : 'northwind_inventory_transactions.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

OperationFailure: bad auth : Authentication failed., full error: {'ok': 0, 'errmsg': 'bad auth : Authentication failed.', 'code': 8000, 'codeName': 'AtlasError'}

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [6]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "invoices"

df_invoices = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_invoices.head(2)

InvalidURI: The empty string is not valid username.

In [7]:
# TODO: Extract data from the "Suppliers" collection
client = get_mongo_client(**mongodb_args)

query = {}
collection = "suppliers"

df_suppliers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_suppliers.head(2)

InvalidURI: The empty string is not valid username.

#### 1.2. Lookup the Invoice Date Keys from the Date Dimension Table.
Here we see an example of where a dimension cross-references another dimension; the Date dimension.  The Date dimension is a classic example of a **Role-Playing dimension**. Dates in-and-of themselves are universal; however, when applied to specific events they take on the identity of those events. Here, the *dim_date* table takes on the identity of *dim_invoice_date* to supply invoice date keys to the *dim_invoices* table.

##### 1.2.1. Get the Data from the Date Dimension Table.
First, fetch the Surrogate Primary Key (date_key) and the Business Key (full_date) from the Date Dimension table using the **get_sql_dataframe()** function. Be certain to cast the **full_date** column to the **datetime64[ns]** data type using the **.astype()** function that is native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [19]:
sql_dim_date = "SELECT date_key, full_date FROM northwind_dw2.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'northwind_dw2.dim_date' doesn't exist")
[SQL: SELECT date_key, full_date FROM northwind_dw2.dim_date;]
(Background on this error at: https://sqlalche.me/e/14/f405)

##### 1.2.2. Lookup the Surrogate Primary Key (date_key) that Corresponds to the invoice_date Column

In [ ]:
df_dim_invoice_date = df_dim_date.rename(columns={"date_key" : "invoice_date_key", "full_date" : "invoice_date"})
df_invoices.invoice_date = df_invoices.invoice_date.astype('datetime64[ns]').dt.date
df_invoices = pd.merge(df_invoices, df_dim_invoice_date, on='invoice_date', how='left')
df_invoices.drop(['invoice_date'], axis=1, inplace=True)
df_invoices.head(2)

#### 1.3. Perform Any Necessary Transformations to the DataFrames

In [ ]:
# 1. Rename the "id" column to reflect the entity as it will serve as the business key for lookup operations
df_invoices.rename(columns={"id":"invoice_id"}, inplace=True)

# 2. Since there are no values in the 'due_date' column go ahead and drop it.
df_invoices.drop(['due_date'], axis=1, inplace=True)

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_invoices.insert(0, "invoice_key", range(1, df_invoices.shape[0]+1))
df_invoices.head(2)

In [ ]:
# 1. TODO: Rename the "id" column to reflect the entity as it will serve as the business key for lookup operations
df_suppliers.rename(columns={"id":"supplier_id"}, inplace=True)
# 2. TODO: Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_suppliers.insert(0, "supplier_key", range(1, df_suppliers.shape[0] + 1))
df_suppliers.head(2)

#### 1.3. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

Here we will call our **set_dataframe( )** function to create each dimension table. This function expects a number of parameters including the usual connection information (e.g., user_id, password, MySQL server name and database), the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, the *name* of the column we wish to designate as the *primary_key* column, and finally, the database operation (insert or update). 

In [ ]:
dataframe = df_invoices
table_name = 'dim_invoices'
primary_key = 'invoice_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [ ]:
# TODO: Upload the "Suppliers" dataframe to create the new "dim_suppliers" dimension table
dataframe = df_suppliers
table_name = "dim_invoices"
primary_key = 'invoice_key'
db_opertaion = 'insert'

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 1.4. Validate that the New Dimension Tables were Created.

In [ ]:
sql_invoices = "SELECT * FROM northwind_dw2.dim_invoices;"
df_dim_invoices = get_sql_dataframe(sql_invoices, **mysql_args)
df_dim_invoices.head(2)

In [ ]:
# TODO: Validate the new "dim_suppliers" table in the northwind_dw2 data warehouse.
sql_suppliers = "SELECT * FROM northwind_dw2.dim_suppliers;"
df_dim_suppliers = get_sql_dataframe(sql_suppliers, **mysql_args)
df_dim_suppliers.head(2)

### 2.0. Create and Populate the New Fact Tables
#### 2.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [ ]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "purchase_orders"

df_fact_pos = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_fact_pos.head(2)

In [ ]:
# TODO: Extract data for your new "Inventory Transactions" Fact Table
query = {}

collection = "inventory_transactions"

df_fact_pos = get_mongo_dataframe(conn_str['atlas'], src_dbname, collection, query)
df_fact_pos.head(2)

#### 2.2. Lookup the DateKeys from the Date Dimension Table.
**2.2.1.** For each date typed column in the **purchase orders** fact table, lookup the corresponding Surrogate Primary Key column. Be certain to cast the date typed column to the **datetime64[ns]** data type using the **.astype()** function that is native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [ ]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "submitted_date" Column.
df_dim_submitted_date = df_dim_date.rename(columns={"date_key" : "submitted_date_key", "full_date" : "submitted_date"})
df_fact_pos.submitted_date = df_fact_pos.submitted_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_dim_submitted_date, on='submitted_date', how='left')
df_fact_pos.drop(['submitted_date'], axis=1, inplace=True)
df_fact_pos.head(2)

In [ ]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "creation_date" Column.
df_dim_creation_date = df_dim_date.rename(columns={"date_key" : "creation_date_key", "full_date" : "creation_date"})
df_fact_pos.creation_date = df_fact_pos.creation_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_creation_date_date, on='creation_date', how='left')
df_fact_pos.drop(['creation_date'], axis=1, inplace=True)
df_fact_pos.head(2)

In [ ]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "po_detail_date_received" Column.
df_dim_po_detail_date_received = df_dim_date.rename(columns={"date_key" : "po_detail_date_received_key", "full_date" : "po_detail_date_received"})
df_fact_pos.po_detail_date_received = df_fact_pos.po_detail_date_received.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_po_detail_date_received, on='po_detail_date_received', how='left')
df_fact_pos.drop(['po_detail_date_received'], axis=1, inplace=True)
df_fact_pos.head(2)

In [ ]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "approved_date" Column.
df_dim_approved_date = df_dim_date.rename(columns={"date_key" : "approved_date_key", "full_date" : "approved_date"})
df_fact_pos.approved_date = df_fact_pos.approved_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_dim_approved_date, on='approved_date', how='left')
df_fact_pos.drop(['approved_date'], axis=1, inplace=True)
df_fact_pos.head(2)

**2.2.2.** Then, for each date typed column in the **inventory transaction** fact table, lookup the corresponding Surrogate Primary Key column. Be certain to cast the date typed column to the **datetime64[ns]** data type using the **.astype()** function, and extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [ ]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "transaction_created_date" Column.
df_dim_transaction_created_date = df_dim_transaction_created_date.rename(columns={"date_key" : "transaction_created_date_key", "full_date" : "transaction_created_date"})
df_fact_pos.transaction_created_date = df_fact_pos.transaction_created_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_dim_transaction_created_date, on='transaction_created_date', how='left')
df_fact_pos.drop(['transaction_created_date'], axis=1, inplace=True)
df_fact_pos.head(2)

df_fact_inventory.head(2)

In [ ]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "transaction_modified_date" Column.
df_dim_transaction_modified_date = df_dim_transaction_modified_date.rename(columns={"date_key" : "transaction_modified_date_key", "full_date" : "transaction_modified_date"})
df_fact_pos.transaction_modified_date = df_fact_pos.transaction_modified_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_transaction_modified_date, on='transaction_modified_date', how='left')
df_fact_pos.drop(['transaction_modified_date'], axis=1, inplace=True)
df_fact_pos.head(2)

df_fact_inventory.head(2)

#### 2.3. Lookup the Primary Keys from the Dimension Tables
**Foreign key relationships** must be established between each newly-crafted **Fact table** and each related **Dimension table**.

##### 2.3.1. First, fetch the Surrogate Primary Key and the Business Key from each Dimension table.

In [ ]:
sql_dim_invoices = "SELECT invoice_key, invoice_id FROM northwind_dw2.dim_invoices;"
df_dim_invoices = get_sql_dataframe(sql_dim_invoices, **mysql_args)
df_dim_invoices.head(2)

In [ ]:
# TODO: Extract the 'primary key' and the 'business key' from your new "dim_suppliers" dimension table
sql_dim_suppliers = "SELECT supplier_key, supplier_id FROM northwind_dw2.dim_supplier;"
df_dim_supplier = get_sql_dataframe(sql_dim_supplier, **mysql_args)
df_dim_invoices.head(2)

In [ ]:
# TODO: Extract the 'primary key' and the 'business key' from the "dim_employees" dimension table
sql_dim_employees = "SELECT employee_key, employee_id FROM northwind_dw2.dim_employees;"
df_dim_employees = get_sql_dataframe(sql_dim_employees, **mysql_args)
df_dim_employees.head(2)

In [ ]:
# TODO: Extract the 'primary key' and the 'business key' from the "dim_products" dimension table
sql_dim_products = "SELECT product_key, product_id FROM northwind_dw2.dim_product;"
df_dim_products = get_sql_dataframe(sql_dim_product, **mysql_args)
df_dim_products.head(2)

##### 2.3.2. Next, use Business Keys to lookup corresponding Surrogate Primary Keys in the Dimension tables
**2.3.2.1. Inventory Fact table.**  The inventory fact table contains a reference to the *dim_products* table by way of the *product_id* column that enables us to lookup the corresponding *product_key*.

In [ ]:
# TODO: Merge the "fact_inventory" and "dim_products" dataframes on the 'product_id' to
# get the 'product_key'. Then drop the 'product_id' column and display the results.
df_fact_products = pd.merge(fact_inventory, dim_products, on='product_id', how='inner')
df_fact_products.drop(['product_id'], axis=1, inplace=True)
df_fact_products.head(2)

In [ ]:
# TODO: Insert a new 'fact_inventory_transaction_key' column, with an ever-incrementing
# numeric value, to serve as the surrogate primary key. Then display the results.
df_fact_products.insert(0, "fact_inventory_transaction_key", range(1, df_fact_products.shape[0]+1))
df_fact_products.head(2)

**2.3.2.2. Purchase Order Fact table**. Here we see another example of a **role-playing dimension**; the *Employee* dimension. The employee dimension is another classic example of a Role-Playing dimension. In-and-of themselves Employees are generic; however, when applied to specific roles they take on the identity of those positions (roles). Here, the *dim_employee* table takes on the identities of *creator, approver, and submitter* in supplying employee keys to the *dim_fact_pos* table.

We also see an example of where one fact table is referenced by another fact table as if it were a dimension table; i.e., a **fact dimension.** In this instance the purchase orders table *fact_pos* contains a reference to the inventory transaction table *fact_inventory* by way of the *inventory_transaction_id* column, enabling us to lookup the corresponding *fact_inventory_key*.

In [ ]:
# Lookup the Surrogate Primary Key (employee_key) that Corresponds to the "created_by" Column.
df_dim_creator = df_dim_employees.rename(columns={"employee_key":"created_by_key", "employee_id":"created_by"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_creator, on='created_by', how='inner')
df_fact_pos.drop(['created_by'], axis=1, inplace=True)
df_fact_pos.head(2)

In [ ]:
# TODO: Lookup the Surrogate Primary Key (employee_key) that Corresponds to the "approved_by" Column.
df_dim_approver = df_dim_employees.rename(columns={"employee_key":"approved_by_key", "employee_id":"approved_by"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_approver, on='approved_by', how='inner')
df_fact_pos.drop(['approved_by'], axis=1, inplace=True)
df_fact_pos.head(2)

In [ ]:
# TODO: Lookup the Surrogate Primary Key (employee_key) that Corresponds to the "submitted_by" Column.
df_dim_submitter = df_dim_employees.rename(columns={"employee_key":"submitted_by_key", "employee_id":"submitted_by"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_submitter, on='submitted_by', how='inner')
df_fact_pos.drop(['submitted_by'], axis=1, inplace=True)
df_fact_pos.head(2)

Since the *df_fact_pos* fact table contains references to suppliers and products, lookup the surrogate primary keys from the *dim_suppliers* and *dim_products* tables that correspond to the *supplier_id* and *product_id* columns.

In [ ]:
# TODO: Lookup the Surrogate Primary Key (supplier_key) that Corresponds to the "supplier_id" Column.
df_dim_supplier = df_dim_supplier.rename(columns={"suppplier_key":"supplier_id_key", "employee_id":"suppplier_id"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_supplier, on='supplier_id', how='inner')
df_fact_pos.drop(['supplier_id'], axis=1, inplace=True)
df_fact_pos.head(2)

In [ ]:
# TODO: Lookup the Primary Key (product_key) that Corresponds to the "po_detail_product_id" Column.
# Remember that you'll first have to rename it to "product_id" in the "df_fact_pos" dataframe.
df_dim_products = df_dim_employees.rename(columns={"product_key":"po_detail_product_id_key", "employee_id":"po_detail_supplier_id"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_products, on='po_detail_product_id', how='inner')
df_fact_pos.drop(['po_detail_product_idy'], axis=1, inplace=True)
df_fact_pos.head(2)

In [ ]:
# TODO: Insert a new "fact_purchase_order_key" column, with an ever-incrementing 
# numeric value, to serve as the surrogate primary key. Then display the results.
df_fact_pos.insert(0, "fact_purchase_order_key", range(1, df_fact_pos.shape[0]+1)))

Since the purchase orders table *fact_pos* contains a reference to the inventory transaction table *fact_inventory* by way of the *inventory_transaction_id* column, we can lookup the corresponding *fact_inventory_transaction_key*. In this case, the *fact_inventory* table plays the role of a dimension table. This is called a **fact dimension**.

In [ ]:
# The inventory fact table acts as a Fact Dimension: po_detail_inventory_id
df_dim_fact_inventory = df_fact_inventory[['fact_inventory_transaction_key','inventory_transaction_id']]
df_fact_pos.rename(columns={"po_detail_inventory_id":"inventory_transaction_id"}, inplace=True)
df_fact_pos = pd.merge(df_fact_pos, df_dim_fact_inventory, on='inventory_transaction_id', how='left')
df_fact_pos.drop(['inventory_transaction_id'], axis=1, inplace=True)
df_fact_pos.head(2)

#### 2.4. Perform Any Necessary Transformations to the DataFrames

In [ ]:
# Rename the Foreign Key Columns
column_name_map = {"transaction_created_date_key" : "created_date_key",
                   "transaction_modified_date_key" : "modified_date_key",
                  }

df_fact_inventory.rename(columns=column_name_map, inplace=True)

# Reorder the Columns
ordered_columns = ['fact_inventory_transaction_key','inventory_transaction_id'
                   ,'inventory_transaction_type','product_key'
                   ,'created_date_key','modified_date_key','quantity']

df_fact_inventory = df_fact_inventory[ordered_columns]
df_fact_inventory.head(2)

In [ ]:
# Change the 'po_detail_posted_to_inventory' column values from 1 and 0 to 'True' and 'False'.
df_fact_pos['posted_to_inventory'] = df_fact_pos.po_detail_posted_to_inventory.map({1 :'True', 0 :'False'})
df_fact_pos.drop('po_detail_posted_to_inventory', axis=1, inplace=True)
df_fact_pos.rename(columns={'posted_to_inventory' : 'po_detail_posted_to_inventory'}, inplace=True)

# Reorder the Columns
reordered_columns = ['fact_purchase_order_key','purchase_order_id','supplier_key','product_key'
                     ,'fact_inventory_transaction_key','created_by_key','creation_date_key'
                     ,'submitted_by_key','submitted_date_key','approved_by_key','approved_date_key'
                     ,'purchase_order_status','po_detail_date_received_key'
                     ,'po_detail_posted_to_inventory','po_detail_unit_cost','po_detail_quantity'
                     ,'shipping_fee','taxes','payment_date','payment_amount']

df_fact_pos = df_fact_pos[reordered_columns]
df_fact_pos.head(2)

#### 2.5. Load Newly Transformed MongoDB Data into the Northwind_DW2 Data Warehouse

In [ ]:
dataframe = df_fact_pos
table_name = 'fact_purchase_orders'
primary_key = 'fact_purchase_order_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [ ]:
# TODO: Upload the "Inventory Transaction" dataframe to create the new "fact_inventory_transactions" fact table. 
dataframe = df_fact_inventory
table_name = 'fact_inventory_transactions'
primary_key = 'fact_inventory_transaction_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 2.6. Validate that the New Fact Tables were Created

In [ ]:
# TODO: Validate the correctness of the new "Purchase Orders" fact table.
sql_purchase_orders = "SELECT * FROM northwind_dw2.fact_purchase_orders;"
df_fact_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_purchase_orders.head(2)

In [ ]:
# TODO: Validate the correctness of the new "Inventory Transactions" fact table.
sql_inventory_transactions = "SELECT * FROM northwind_dw2.fact_inventory_transactions;"
df_fact_inventory_transactions = get_sql_datafrane(sql_inventory_transactions, **mysql_args)
df_fact_inventory_transactions.head(2)

### 3.0. Demonstrate that the New Data Warehouse Exists and Contains the Correct Data
To demonstrate the viability of your solution, author a SQL SELECT statement that returns:
- Each Company’s Name
- The total amount of the purchase order detail quantity associated with each company
- The total amount of the purchase order detail unit cost associated with each company

**NOTE:** *Remember that a string typed variable whose value is contained by triple-quotes (""" ... """) can preserve multi-line formatting, and that a string variable has an intrinsic **.format()** function that accepts ordered parameters that will replace tokens (e.g., {0}) in the formatted string.*  

In [ ]:
sql_purchase_orders = """
    SELECT s.company AS 'Company'
        , FORMAT(SUM(po.po_detail_quantity),0) AS 'Total Quantity'
        , FORMAT(SUM(po.po_detail_unit_cost),2) AS 'Total Unit Cost'
    FROM northwind_dw2.fact_purchase_orders AS po
    INNER JOIN northwind_dw2.dim_suppliers AS s
    ON po.supplier_key = s.supplier_key
    GROUP BY s.company
    ORDER BY 'Total Unit Cost' DESC;
    
"""

In [ ]:
df_fact_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_purchase_orders

### 3.1 Extra Credit: Author a Query that Returns the Inventory Transaction Data
- Each Product Category
- Each TransactionType
- The total number of transactions associated with each Product Category and Transaction Type
- The total amount (quantity) associated with each Product Category and Transaction Type

In [ ]:
sql_inventory_transactions = """

"""

In [ ]:
df_fact_inventory_transactions = get_sql_dataframe(sql_inventory_transactions, **mysql_args)
df_fact_inventory_transactions